In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 12.6 MB/s 
     |████████████████████████████████| 209 kB 89.4 MB/s 
     |████████████████████████████████| 81 kB 10.3 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
     |████████████████████████████████| 112 kB 60.7 MB/s 
     |████████████████████████████████| 146 kB 72.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=de9625618f87f076cb54ace8ab820d07071b6e17d844e24b5f6bb88ab3746758
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
!nvidia-smi

Wed Jun 29 15:09:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Data

In [ ]:
# For Colab only!
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

In [ ]:
all_data = []
if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "data":
    os.chdir('/content/drive/MyDrive/BA/data')

for f in os.listdir():
    # print(f)
    part_id = f.split("_")[1].split(".")[0]
    # print(part_id)
    df = pd.read_csv(f, sep=";")
    df["part_id"] = part_id
    all_data.append(df)

In [ ]:
print(len(all_data))
all_data[0].head()

60


,timestamp,active,xPos,yPos,TargetX,StartX,TunnelAmplitude,TunnelWidth,YDistanceToTop,YDistanceToBottom,part_id
0,1651594672586,0,0,0,356.5,1563.5,1207,45,-442.0,487.0,95528384
1,1651594672604,0,976,929,356.5,1563.5,1207,45,411.5,-366.5,95528384
2,1651594672620,0,976,929,356.5,1563.5,1207,45,411.5,-366.5,95528384
3,1651594672631,0,976,929,356.5,1563.5,1207,45,411.5,-366.5,95528384
4,1651594672653,0,976,929,356.5,1563.5,1207,45,411.5,-366.5,95528384


# Preprocess Data

## Normalize by Participants Displaysize

In [ ]:
import copy

In [ ]:
def normalize_by_screen(datapoint, screenwidth, screenheight):
    normalized = copy.deepcopy(datapoint)
    normalized["xPos"] /= screenwidth
    normalized["TargetX"] /= screenwidth
    normalized["StartX"] /= screenwidth
    normalized["TunnelAmplitude"] /= screenwidth
    normalized["yPos"] /= screenheight
    normalized["TunnelWidth"] /= screenheight
    normalized["YDistanceToTop"] /= screenheight
    normalized["YDistanceToBottom"] /= screenheight
    return normalized

In [ ]:
def get_participants_screensize(datapoint):
    x_res = -1
    y_res = -1
    # print(datapoint)
    curr_part_id = datapoint["part_id"][0]
    # print(part_id)
    for f in os.listdir():
        curr_id = f.split("_")[1].split(".")[0]
        if curr_id == curr_part_id:
            demographic = pd.read_csv(f, sep=";")
            real_res_string = demographic["Real Res"][0]
            real_res_string = real_res_string.replace("px", "")
            x_res = real_res_string.split(" x ")[0]
            y_res = real_res_string.split(" x ")[1]
    return int(x_res), int(y_res)

In [ ]:
def normalize_screensize(dataset):
    if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "demographic_data":
        os.chdir("/content/drive/MyDrive/BA/demographic_data")
    for i, datapoint in enumerate(dataset):
        screenwidth, screenheight = get_participants_screensize(datapoint)
        dataset[i] = normalize_by_screen(datapoint, screenwidth, screenheight)

## First Order Derivation
The goal is to make the movement independent of the positions
Therefore, xPos and yPos are converted to speeds by taking the first derivation

In [ ]:
def get_data_with_derivate(datapoint):
    diffSeries = datapoint["xPos"].diff(periods=1)
    datapoint.insert(3, "xSpeed", diffSeries)
    diffSeries = datapoint["yPos"].diff(periods=1)
    datapoint.insert(5, "ySpeed", diffSeries)

## Umwandeln in Array

In [ ]:
import numpy as np

In [ ]:
def get_np_arrays(data):
    data_array = []
    for curr in data:
        removed = copy.deepcopy(curr)
        removed = removed.drop('timestamp', axis=1)
        removed = removed.drop('xPos', axis=1)
        removed = removed.drop('yPos', axis=1)
        removed = removed.drop('part_id', axis=1)
        data_array.append(removed.to_numpy(copy=True))
    return data_array

## Split into sections of n samples

In [ ]:
def append_samples_of_length_n(participant_data, n, samples, labels, label_distance):
    for i in range(participant_data.shape[0] - (n+label_distance)):
        samples.append(copy.deepcopy(participant_data[i:i+n]))
        sum_to_label = np.array([0,0], dtype='float64')
        for j in range (i+n+1, i+n+label_distance):
            sum_to_label += participant_data[j][1:3]
        labels.append(copy.deepcopy(sum_to_label))
        # labels.append(copy.deepcopy(participant_data[i+n+label_distance][1:3]))

In [ ]:
def get_samples_and_labels(data_array, N=10, label_distance=10):
    samples = []
    labels = []
    for participant_data in data_array:
        append_samples_of_length_n(participant_data, N, samples, labels, label_distance)
    return samples, labels

## Remove unusable samples
If in one sample all 10 xSpeeds and ySpeeds are equal the sample is removed, if between the last sample and the label the mouse isn't moved the sample is removed

In [ ]:
def remove_unusable(data, labels, verbose=0):
    # print(labels)
    new_data = []
    new_labels = []
    for i, sample in enumerate(data):
        bool_array = np.all(sample == sample[0,:], axis=0)
        if bool_array[1] == False or bool_array[2] == False:
            if labels[i][0] != 0:
                new_data.append(sample)
                new_labels.append(labels[i])
        if verbose > 1 and i % 100000 == 0:
            print(f'evaluated {i}')

    if verbose > 0:
        print(f'removed {len(data) - len(new_data)} samples')

    return new_data, new_labels

## Make all Samples face the same way
The average xSpeed is calculated, if this average is negative all xSpeeds are multiplied by one to make all vectors face in the same direction

In [ ]:
def make_same_orientation(data, labels, verbose=0):
    changed = 0
    for i, sample in enumerate(data):
        average_x_speed = np.average(sample, axis=0)[1]
        if average_x_speed < 0:
            sample[:, 1] *= -1
            labels[i][0] *= -1
            if verbose > 0:
                changed += 1
    if verbose > 0:
        print(f'Changed {changed} samples')

## Transform a dataset

In [ ]:
def get_transformed_data(dataset, N, label_distance, verbose):
    normalize_screensize(dataset)

    if'xSpeed' not in dataset[0].columns:
        for datapoint in dataset:
            get_data_with_derivate(datapoint)
            datapoint.drop(index=0, axis=0, inplace=True)

    data_array = get_np_arrays(dataset)

    samples, labels = get_samples_and_labels(data_array, N, label_distance)

    samples, labels = remove_unusable(samples, labels, verbose=verbose)

    make_same_orientation(samples, labels, verbose=verbose)

    return samples, labels

## Train-Test-Split
Initially the entire dataset given in all_data are split into Test and Train group by participants, than the datasets are preprocessed in the same way

In [ ]:
TRAIN_SPLIT = 0.8
TEST_SPLIT = 1.0 - TRAIN_SPLIT

In [ ]:
import random as rand
SEED = 27255

In [ ]:
def get_train_test_split_data(dataset):
    train_data_amount = 0.8 * len(dataset)

    train_data = []
    test_data = []

    rand.seed(SEED)
    train_indizes = rand.sample(range(len(dataset)), int(train_data_amount))
    # print(train_indizes)

    for i, sample in enumerate(dataset):
        if i in train_indizes:
            train_data.append(sample)
        else:
            test_data.append(sample)

    # print(len(train_data))
    # print(len(test_data))

    return train_data, test_data

## Normalize Trainingdata
The average column values of the training data is calculated, each value in test and training data is divided by the average of its column

In [ ]:
def get_average_array(arr, axis=(0,1)):
    return np.mean(arr, axis=axis)

In [ ]:
def get_normalized_arrays(train_samples, train_labels, test_samples, test_labels):
    #avg_array = get_average_array(train_samples)
    #avg_label_array = get_average_array(train_labels, 0)

    train_array = np.array(train_samples)
    test_array = np.array(test_samples)
    train_labels_array = np.array(train_labels)
    test_labels_array = np.array(test_labels)

    #train_normalized = train_array[:, :, ] / avg_array
    #train_normalized[:, : , 0] = train_array[:, :, 0]
    #train_normalized[:, :, 2] = train_array[:, :, 2]

    #test_normalized = test_array[:, :, ] / avg_array
    #test_normalized[:, :, 0] = test_array[:, :, 0]
    #test_normalized[:, :, 2] = test_array[:, :, 2]

    # train_labels_array[:, 0] /= avg_label_array[0:1] - averages are very low!

    # test_labels_array[:, 0] /= avg_label_array[0:1]

    return train_array, train_labels_array, test_array, test_labels_array

## Shuffle Arrays

In [ ]:
from sklearn.utils import shuffle

In [ ]:
VERBOSE = 1
LABEL_DISTANCE = 6 # 6 => 100ms

def get_preprocessed_data(N):
    curr_data = copy.deepcopy(all_data)
    train_data, test_data = get_train_test_split_data(curr_data)

    train_samples, train_labels = get_transformed_data(train_data, N, LABEL_DISTANCE, 0)
    test_samples, test_labels = get_transformed_data(test_data, N, LABEL_DISTANCE, 0)

    train_samples, train_labels, test_samples, test_labels = get_normalized_arrays(train_samples, train_labels, test_samples, test_labels)

    train_samples, train_labels = shuffle(train_samples, train_labels, random_state=SEED)
    test_samples, test_labels = shuffle(test_samples, test_labels, random_state=SEED)

    return train_samples, train_labels, test_samples, test_labels

# Hyperparameter Optimization

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import optuna

In [ ]:
OPTIMIZATION_SPLIT = 0.2
NUMBER_OF_TRIALS = 100
EPOCHS = 20

In [ ]:
def noiseToSignalLoss(y_true, y_pred):
    losses = tf.math.divide(
        tf.math.reduce_sum(
            tf.math.pow(
               tf.math.subtract(y_true, y_pred)
            , 2)
        , axis=-1),
        tf.math.reduce_sum(
            tf.math.pow(y_true, 2)
        , axis=-1)
    )

    return tf.reduce_mean(losses, axis=-1)

# Final Optimization - Dense for MAE

In [ ]:
def create_dataset(trial):
    N = trial.suggest_int('sequence_length', 3, 7)

    train_samples, train_labels, test_samples, test_labels = get_preprocessed_data(N)

    highest_index_train = int(train_samples.shape[0] * OPTIMIZATION_SPLIT)
    highest_index_test = int(test_samples.shape[0] * OPTIMIZATION_SPLIT)

    train_samples = train_samples.reshape(train_samples.shape[0], train_samples.shape[1] * train_samples.shape[2])
    test_samples = test_samples.reshape(test_samples.shape[0], test_samples.shape[1] * test_samples.shape[2])

    return train_samples[:highest_index_train], train_labels[:highest_index_train], test_samples[:highest_index_test], test_labels[:highest_index_test]

In [ ]:
def create_model(trial, train_X):
    number_of_inputs = train_X.shape[1]

    ff1_hidden_neurons = trial.suggest_int('ff1_hidden_neurons', 70, 250)
    ff2_hidden_neurons = trial.suggest_int('ff2_hidden_neurons', 70, 250)
    ff3_hidden_neurons = trial.suggest_int('ff3_hidden_neurons', 70, 250)
    ff4_hidden_neurons = trial.suggest_int('ff4_hidden_neurons', 70, 250)
    ff5_hidden_neurons = trial.suggest_int('ff5_hidden_neurons', 70, 250)
    ff6_hidden_neurons = trial.suggest_int('ff6_hidden_neurons', 70, 250)
    ff7_hidden_neurons = trial.suggest_int('ff7_hidden_neurons', 70, 250)
    ff8_hidden_neurons = trial.suggest_int('ff8_hidden_neurons', 70, 250)
    ff9_hidden_neurons = trial.suggest_int('ff9_hidden_neurons', 70, 250)
    ff10_hidden_neurons = trial.suggest_int('ff10_hidden_neurons', 70, 250)
    ff11_hidden_neurons = trial.suggest_int('ff11_hidden_neurons', 70, 250)
    ff12_hidden_neurons = trial.suggest_int('ff12_hidden_neurons', 70, 250)
    ff13_hidden_neurons = trial.suggest_int('ff13_hidden_neurons', 70, 250)
    ff14_hidden_neurons = trial.suggest_int('ff14_hidden_neurons', 70, 250)
    ff15_hidden_neurons = trial.suggest_int('ff15_hidden_neurons', 70, 250)
    ff16_hidden_neurons = trial.suggest_int('ff16_hidden_neurons', 70, 250)
    ff17_hidden_neurons = trial.suggest_int('ff17_hidden_neurons', 70, 250)

    model = keras.Sequential()
    model.add(
        layers.Input(shape=(number_of_inputs, ))
    )

    model.add(
        layers.Dense(ff1_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff2_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff3_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff4_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff5_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff6_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff7_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff8_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff9_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff10_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff11_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff12_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff13_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff14_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff15_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff16_hidden_neurons, activation='tanh')
    )

    model.add(
        layers.Dense(ff17_hidden_neurons, activation='tanh')
    )
    
    model.add(layers.Dropout(0.2296))

    model.add(layers.Dense(2, activation='linear'))

    return model

In [ ]:
def objective(trial):
    train_X, train_Y, test_X, test_Y = create_dataset(trial)
    model = create_model(trial, train_X)

    monitor = 'val_loss'

    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10),
        optuna.integration.TFKerasPruningCallback(trial, monitor),
    ]

    lr = trial.suggest_float('learning_rate', 1e-10, 1e-4, log=True)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss=keras.losses.MeanAbsoluteError(), metrics=[keras.metrics.MeanSquaredError(), keras.metrics.RootMeanSquaredError(), noiseToSignalLoss])

    history = model.fit(train_X, train_Y, batch_size=382, validation_split=0.1, verbose=1, epochs=EPOCHS, callbacks=callbacks)

    eval_results = model.evaluate(test_X, test_Y, batch_size=382)

    return float(eval_results[0])

In [ ]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

study.optimize(objective, n_trials=NUMBER_OF_TRIALS)

# show results
pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print('Study statistics:')
print('   Number of finished Trials: ', len(study.trials))
print('   Number of pruned Trials: ', len(pruned_trials))
print('   Number of complete Trials: ', len(complete_trials))

print('Best Trial: ')
trial = study.best_trial

print('  MAE: ', trial.value)

print('   Params: ')
for key, value in trial.params.items():
    print('   {}: {}'.format(key, value))

[I 2022-06-29 15:33:58,407] A new study created in memory with name: no-name-8c0ceb91-ddaf-49ad-8315-5a764dd1c82d


Epoch 1/20
800/800 [==============================] - 8s 5ms/step - loss: 0.0454 - mean_squared_error: 0.0048 - root_mean_squared_error: 0.0695 - noiseToSignalLoss: 527.1057 - val_loss: 0.0179 - val_mean_squared_error: 0.0021 - val_root_mean_squared_error: 0.0460 - val_noiseToSignalLoss: 38.4445
Epoch 2/20
800/800 [==============================] - 4s 5ms/step - loss: 0.0217 - mean_squared_error: 0.0022 - root_mean_squared_error: 0.0473 - noiseToSignalLoss: 65.2406 - val_loss: 0.0156 - val_mean_squared_error: 0.0020 - val_root_mean_squared_error: 0.0446 - val_noiseToSignalLoss: 22.3817
Epoch 3/20
800/800 [==============================] - 4s 5ms/step - loss: 0.0178 - mean_squared_error: 0.0020 - root_mean_squared_error: 0.0449 - noiseToSignalLoss: 34.9820 - val_loss: 0.0149 - val_mean_squared_error: 0.0020 - val_root_mean_squared_error: 0.0442 - val_noiseToSignalLoss: 17.9982
Epoch 4/20
800/800 [==============================] - 4s 5ms/step - loss: 0.0165 - mean_squared_error: 0.0019 -

[I 2022-06-29 15:38:10,726] Trial 0 finished with value: 0.01036657951772213 and parameters: {'sequence_length': 7, 'ff1_hidden_neurons': 155, 'ff2_hidden_neurons': 173, 'ff3_hidden_neurons': 137, 'ff4_hidden_neurons': 89, 'ff5_hidden_neurons': 169, 'ff6_hidden_neurons': 90, 'ff7_hidden_neurons': 99, 'ff8_hidden_neurons': 96, 'ff9_hidden_neurons': 249, 'ff10_hidden_neurons': 167, 'ff11_hidden_neurons': 154, 'ff12_hidden_neurons': 217, 'ff13_hidden_neurons': 172, 'ff14_hidden_neurons': 233, 'ff15_hidden_neurons': 186, 'ff16_hidden_neurons': 70, 'ff17_hidden_neurons': 225, 'learning_rate': 9.233281220046108e-07}. Best is trial 0 with value: 0.01036657951772213.


Epoch 1/20
756/756 [==============================] - 5s 5ms/step - loss: 0.1048 - mean_squared_error: 0.0200 - root_mean_squared_error: 0.1415 - noiseToSignalLoss: 2780.5178 - val_loss: 0.0193 - val_mean_squared_error: 0.0022 - val_root_mean_squared_error: 0.0466 - val_noiseToSignalLoss: 71.5738
Epoch 2/20
756/756 [==============================] - 4s 5ms/step - loss: 0.0596 - mean_squared_error: 0.0067 - root_mean_squared_error: 0.0816 - noiseToSignalLoss: 775.8151 - val_loss: 0.0177 - val_mean_squared_error: 0.0021 - val_root_mean_squared_error: 0.0459 - val_noiseToSignalLoss: 52.4508
Epoch 3/20
756/756 [==============================] - 4s 5ms/step - loss: 0.0440 - mean_squared_error: 0.0042 - root_mean_squared_error: 0.0647 - noiseToSignalLoss: 378.7591 - val_loss: 0.0169 - val_mean_squared_error: 0.0021 - val_root_mean_squared_error: 0.0454 - val_noiseToSignalLoss: 38.3442
Epoch 4/20
756/756 [==============================] - 4s 6ms/step - loss: 0.0345 - mean_squared_error: 0.003

[I 2022-06-29 15:41:16,290] Trial 1 finished with value: 0.011651416309177876 and parameters: {'sequence_length': 3, 'ff1_hidden_neurons': 194, 'ff2_hidden_neurons': 115, 'ff3_hidden_neurons': 219, 'ff4_hidden_neurons': 101, 'ff5_hidden_neurons': 211, 'ff6_hidden_neurons': 98, 'ff7_hidden_neurons': 91, 'ff8_hidden_neurons': 99, 'ff9_hidden_neurons': 148, 'ff10_hidden_neurons': 217, 'ff11_hidden_neurons': 216, 'ff12_hidden_neurons': 159, 'ff13_hidden_neurons': 79, 'ff14_hidden_neurons': 206, 'ff15_hidden_neurons': 248, 'ff16_hidden_neurons': 169, 'ff17_hidden_neurons': 161, 'learning_rate': 2.567403779135952e-07}. Best is trial 0 with value: 0.01036657951772213.


...

Epoch 1/20
785/785 [==============================] - 5s 5ms/step - loss: 0.0154 - mean_squared_error: 0.0019 - root_mean_squared_error: 0.0435 - noiseToSignalLoss: 38.4320 - val_loss: 0.0130 - val_mean_squared_error: 0.0017 - val_root_mean_squared_error: 0.0416 - val_noiseToSignalLoss: 22.8572
Epoch 2/20
785/785 [==============================] - 4s 5ms/step - loss: 0.0127 - mean_squared_error: 0.0017 - root_mean_squared_error: 0.0409 - noiseToSignalLoss: 22.5648 - val_loss: 0.0126 - val_mean_squared_error: 0.0017 - val_root_mean_squared_error: 0.0414 - val_noiseToSignalLoss: 20.9328
Epoch 3/20
785/785 [==============================] - 4s 5ms/step - loss: 0.0124 - mean_squared_error: 0.0016 - root_mean_squared_error: 0.0404 - noiseToSignalLoss: 21.5347 - val_loss: 0.0123 - val_mean_squared_error: 0.0017 - val_root_mean_squared_error: 0.0409 - val_noiseToSignalLoss: 13.5187
Epoch 4/20
785/785 [==============================] - 4s 5ms/step - loss: 0.0122 - mean_squared_error: 0.0016 - 

[I 2022-06-29 19:04:36,542] Trial 98 pruned. Trial was pruned at epoch 4.


Epoch 1/20
756/756 [==============================] - 5s 5ms/step - loss: 0.0281 - mean_squared_error: 0.0032 - root_mean_squared_error: 0.0568 - noiseToSignalLoss: 227.5069 - val_loss: 0.0149 - val_mean_squared_error: 0.0020 - val_root_mean_squared_error: 0.0446 - val_noiseToSignalLoss: 15.3512
Epoch 2/20
750/756 [============================>.] - ETA: 0s - loss: 0.0159 - mean_squared_error: 0.0019 - root_mean_squared_error: 0.0437 - noiseToSignalLoss: 21.0081

[I 2022-06-29 19:06:29,985] Trial 99 pruned. Trial was pruned at epoch 1.


Study statistics:
   Number of finished Trials:  100
   Number of pruned Trials:  92
   Number of complete Trials:  8
Best Trial: 
  MAE:  0.009092113934457302
   Params: 
   sequence_length: 6
   ff1_hidden_neurons: 118
   ff2_hidden_neurons: 233
   ff3_hidden_neurons: 108
   ff4_hidden_neurons: 225
   ff5_hidden_neurons: 188
   ff6_hidden_neurons: 205
   ff7_hidden_neurons: 151
   ff8_hidden_neurons: 185
   ff9_hidden_neurons: 243
   ff10_hidden_neurons: 174
   ff11_hidden_neurons: 89
   ff12_hidden_neurons: 88
   ff13_hidden_neurons: 227
   ff14_hidden_neurons: 241
   ff15_hidden_neurons: 140
   ff16_hidden_neurons: 173
   ff17_hidden_neurons: 105
   learning_rate: 9.90350364955544e-05


In [ ]:
study.optimize(objective, n_trials=NUMBER_OF_TRIALS)

# show results
pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print('Study statistics:')
print('   Number of finished Trials: ', len(study.trials))
print('   Number of pruned Trials: ', len(pruned_trials))
print('   Number of complete Trials: ', len(complete_trials))

print('Best Trial: ')
trial = study.best_trial

print('  MAE: ', trial.value)

print('   Params: ')
for key, value in trial.params.items():
    print('   {}: {}'.format(key, value))

Epoch 1/20
800/800 [==============================] - 5s 5ms/step - loss: 0.0454 - mean_squared_error: 0.0056 - root_mean_squared_error: 0.0745 - noiseToSignalLoss: 660.2209 - val_loss: 0.0162 - val_mean_squared_error: 0.0020 - val_root_mean_squared_error: 0.0442 - val_noiseToSignalLoss: 29.9788
Epoch 2/20
798/800 [============================>.] - ETA: 0s - loss: 0.0201 - mean_squared_error: 0.0021 - root_mean_squared_error: 0.0455 - noiseToSignalLoss: 55.3023

[I 2022-06-29 19:09:13,068] Trial 100 pruned. Trial was pruned at epoch 1.


Epoch 1/20
793/793 [==============================] - 5s 5ms/step - loss: 0.0172 - mean_squared_error: 0.0021 - root_mean_squared_error: 0.0458 - noiseToSignalLoss: 75.8163 - val_loss: 0.0126 - val_mean_squared_error: 0.0016 - val_root_mean_squared_error: 0.0401 - val_noiseToSignalLoss: 24.8317
Epoch 2/20
793/793 [==============================] - 4s 5ms/step - loss: 0.0133 - mean_squared_error: 0.0017 - root_mean_squared_error: 0.0412 - noiseToSignalLoss: 22.3458 - val_loss: 0.0124 - val_mean_squared_error: 0.0016 - val_root_mean_squared_error: 0.0394 - val_noiseToSignalLoss: 37.0393
Epoch 3/20
793/793 [==============================] - 4s 5ms/step - loss: 0.0127 - mean_squared_error: 0.0017 - root_mean_squared_error: 0.0407 - noiseToSignalLoss: 22.4951 - val_loss: 0.0125 - val_mean_squared_error: 0.0015 - val_root_mean_squared_error: 0.0391 - val_noiseToSignalLoss: 42.3728
Epoch 4/20
793/793 [==============================] - 4s 5ms/step - loss: 0.0125 - mean_squared_error: 0.0016 - 

[I 2022-06-29 19:12:22,107] Trial 101 finished with value: 0.009342661127448082 and parameters: {'sequence_length': 6, 'ff1_hidden_neurons': 117, 'ff2_hidden_neurons': 229, 'ff3_hidden_neurons': 109, 'ff4_hidden_neurons': 235, 'ff5_hidden_neurons': 192, 'ff6_hidden_neurons': 246, 'ff7_hidden_neurons': 140, 'ff8_hidden_neurons': 186, 'ff9_hidden_neurons': 236, 'ff10_hidden_neurons': 169, 'ff11_hidden_neurons': 88, 'ff12_hidden_neurons': 85, 'ff13_hidden_neurons': 223, 'ff14_hidden_neurons': 241, 'ff15_hidden_neurons': 139, 'ff16_hidden_neurons': 191, 'ff17_hidden_neurons': 99, 'learning_rate': 7.674267112444944e-05}. Best is trial 81 with value: 0.009092113934457302.


Epoch 1/20
793/793 [==============================] - 5s 5ms/step - loss: 0.0169 - mean_squared_error: 0.0020 - root_mean_squared_error: 0.0451 - noiseToSignalLoss: 61.6590 - val_loss: 0.0137 - val_mean_squared_error: 0.0017 - val_root_mean_squared_error: 0.0413 - val_noiseToSignalLoss: 20.0717
Epoch 2/20
793/793 [==============================] - ETA: 0s - loss: 0.0133 - mean_squared_error: 0.0017 - root_mean_squared_error: 0.0414 - noiseToSignalLoss: 23.1317

...

Epoch 1/20
793/793 [==============================] - 5s 5ms/step - loss: 0.0189 - mean_squared_error: 0.0022 - root_mean_squared_error: 0.0471 - noiseToSignalLoss: 86.0637 - val_loss: 0.0129 - val_mean_squared_error: 0.0016 - val_root_mean_squared_error: 0.0405 - val_noiseToSignalLoss: 29.8510
Epoch 2/20
791/793 [============================>.] - ETA: 0s - loss: 0.0135 - mean_squared_error: 0.0017 - root_mean_squared_error: 0.0416 - noiseToSignalLoss: 25.5471

[I 2022-06-29 22:39:59,345] Trial 198 pruned. Trial was pruned at epoch 1.


Epoch 1/20
800/800 [==============================] - 5s 5ms/step - loss: 0.0161 - mean_squared_error: 0.0020 - root_mean_squared_error: 0.0442 - noiseToSignalLoss: 54.8441 - val_loss: 0.0125 - val_mean_squared_error: 0.0017 - val_root_mean_squared_error: 0.0412 - val_noiseToSignalLoss: 15.0127
Epoch 2/20
794/800 [============================>.] - ETA: 0s - loss: 0.0127 - mean_squared_error: 0.0016 - root_mean_squared_error: 0.0402 - noiseToSignalLoss: 23.9410

[I 2022-06-29 22:41:55,876] Trial 199 pruned. Trial was pruned at epoch 1.


Study statistics:
   Number of finished Trials:  200
   Number of pruned Trials:  186
   Number of complete Trials:  14
Best Trial: 
  MAE:  0.009016593918204308
   Params: 
   sequence_length: 6
   ff1_hidden_neurons: 134
   ff2_hidden_neurons: 112
   ff3_hidden_neurons: 90
   ff4_hidden_neurons: 223
   ff5_hidden_neurons: 199
   ff6_hidden_neurons: 100
   ff7_hidden_neurons: 145
   ff8_hidden_neurons: 160
   ff9_hidden_neurons: 231
   ff10_hidden_neurons: 83
   ff11_hidden_neurons: 182
   ff12_hidden_neurons: 147
   ff13_hidden_neurons: 165
   ff14_hidden_neurons: 134
   ff15_hidden_neurons: 139
   ff16_hidden_neurons: 244
   ff17_hidden_neurons: 194
   learning_rate: 9.913904350268535e-05
